In [1]:
import numpy as np
from RL import grid_world_mdp
from RL import RL

In [2]:
mdp = grid_world_mdp(4,4,4, [0,15])

In [3]:
rl_obj = RL(mdp)

In [4]:
v, policy = rl_obj.iterative_policy_evaluation()

In [5]:
v

array([  0., -14., -20., -22., -14., -18., -20., -20., -20., -20., -18.,
       -14., -22., -20., -14.,   0.])

In [6]:
policy = policy.reshape((4,4))

In [7]:
rl_obj.get_named_policy(policy)

[['N', 'W', 'W', 'W'],
 ['N', 'N', 'W', 'S'],
 ['N', 'N', 'S', 'S'],
 ['N', 'E', 'E', 'N']]

In [8]:
rl_obj.get_named_policy(policy)

[['N', 'W', 'W', 'W'],
 ['N', 'N', 'W', 'S'],
 ['N', 'N', 'S', 'S'],
 ['N', 'E', 'E', 'N']]